In [1]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [2]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [8,16,32,64,128,256]

In [3]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [4]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [5]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [6]:
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [7]:
df_ma.dropna(inplace=True)

In [8]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
255,2023-06-27T20:00:00.000000000Z,1.09626,1.09634,1.09597,1.09608,1.095989,1.095060,1.093171,1.091361,1.093199,1.090606
256,2023-06-27T21:00:00.000000000Z,1.09604,1.09630,1.09592,1.09609,1.095925,1.095329,1.093321,1.091405,1.093234,1.090675
257,2023-06-27T22:00:00.000000000Z,1.09610,1.09627,1.09564,1.09573,1.096029,1.095493,1.093474,1.091459,1.093277,1.090740
258,2023-06-27T23:00:00.000000000Z,1.09573,1.09586,1.09548,1.09575,1.095997,1.095606,1.093625,1.091597,1.093322,1.090801
259,2023-06-28T00:00:00.000000000Z,1.09576,1.09614,1.09528,1.09548,1.095946,1.095701,1.093768,1.091762,1.093354,1.090859


In [9]:
df_plot = df_ma.iloc[-200:].copy()

In [10]:
df_plot.shape

(200, 11)

In [12]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C',
    name='Price'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()